The following APIs were used in this project  
**mytvsuper.com**  
1.   https://content-api.mytvsuper.com/v1/programme/list?{params}  
Query search for programme  
Filtering and getting the programme ID for other APIs
2.   https://content-api.mytvsuper.com/v1/programme/details?programme_id={programme_id}  
Programme detail from mytvsuper site  
Contains actor list (simplified) & programme categories

**tvb.com**  
3.   https://programme.tvb.com/api/programme/detail?programme_path={programme_path}  
Programme detail from tvb.com  
Contains actor list (detailed) & crew list

4.   https://programme.tvb.com/api/programme/cast?programme_path={programme_path}  
Cast detail from tvb.com  
Contains actor & role list (actors without role record will not be listed)

In order to get a complete actor list, information from API 2,3,4 should be consolidated



In [ ]:
!pip install "requests>=2.31.0"
!pip install "pandas>=1.5.3"

In [ ]:
#import libraries
import requests
import json
import re
import pandas as pd
import unicodedata

Parameters for https://content-api.mytvsuper.com/v1/programme/list

1.   tags[]: tag id for the search query, the meaning of id can be referenced under a programme from  
 https://content-api.mytvsuper.com/v1/programme/details  
 The following tag ids were used
  *   60: 劇集
  *   300: TVB劇集
  *   183: 港劇
  *   30000006/30000007: production year between 2010-2019/2020-2029  
    
  **Tags excluded from results**    
  *   1647: 宣傳片/預告/花絮,
  *   359: 宣傳活動
  *   195: 香港綜藝
  *   313: 主題盛事
  *   381: 明星專訪  
  *   1672: 微電影
  *   354: 娛樂新聞
2.   limit/offset: limit/offset for the search query, set to 10000 (arbitrary large number) & 0
3.   free_episodes_only: limiting the returned programme list to be free episodes only, set to False

In [ ]:
and_tags=[60, 183, 300]
or_tags=[30000006, 30000007]
filter_tags=[359, 1647, 195, 313, 381, 1672, 354]


In [ ]:
programme_id=set()
def fetchlist(tags):
  list_url = "https://content-api.mytvsuper.com/v1/programme/list"
  query = {'offset': '0','limit': '10000', 'free_episodes_only':False}
  query.update({'tags[]':tags})
  return json.loads(requests.get(list_url,query).text)['items']

for or_tag in or_tags:
  tags = and_tags+[or_tag]
  programme_all = fetchlist(tags)

  filter_id = set()
  for filter_tag in filter_tags:
    programme_filter = fetchlist(tags+[filter_tag])

    for f in programme_filter:
      filter_id.add(f['programme_id'])

  for programme in programme_all:
    programme_id.add(programme['programme_id'])
  programme_id=programme_id.difference(filter_id)

print(programme_id)

{123904, 109568, 102405, 118791, 119820, 104972, 122385, 136721, 123923, 131093, 115736, 128025, 102425, 119835, 117788, 135706, 106013, 117791, 101408, 135710, 111136, 110117, 138280, 131624, 131625, 113195, 126510, 126511, 126512, 124978, 120885, 120890, 126522, 115775, 106559, 129602, 139332, 126537, 126538, 132171, 126539, 138829, 110166, 111195, 115804, 115805, 111709, 104031, 113760, 129634, 107109, 118382, 134255, 107119, 124019, 124020, 118901, 128118, 128120, 128121, 133752, 116348, 112252, 117375, 106114, 123011, 104069, 113798, 129159, 137352, 129161, 126089, 125575, 136839, 135816, 135817, 135818, 109712, 135819, 120461, 135820, 120463, 127633, 126102, 116370, 117395, 134802, 128659, 126107, 129180, 126109, 126110, 126111, 126112, 126108, 106657, 123549, 116382, 125599, 115360, 115359, 106664, 106665, 139937, 135842, 106668, 126636, 126127, 126128, 126129, 132271, 113842, 130227, 121524, 125112, 107705, 132796, 104637, 122557, 137922, 113346, 131781, 133831, 133322, 125645,

In [ ]:
print(len(programme_id))

303


Based on the ID list, fetch the programme details fromn mytv API

In [ ]:
def fetchmytvdetails(id):
  details_url = "https://content-api.mytvsuper.com/v1/programme/details"
  query = {'programme_id': id}
  return json.loads(requests.get(details_url,query).text)

mytvdetails=[fetchmytvdetails(x) for x in programme_id]

Even the results are filtered by tags, there are several programmes that are clearly not drama (e.g. one episode only). For simplicity the non-drama records are dropped

In [ ]:
#filtering programmes that only have one episode
mytvdetails_short_eps=[(
    x.get('programme_id')
    ,x.get('name_tc')
    ,x.get('path')
    ,x.get('latest_episode_no')
    ) for x in mytvdetails if x.get('latest_episode_no')==1]

In [ ]:
pd.DataFrame(mytvdetails_short_eps,columns=['id', 'name', 'path', 'eps'])

,id,name,path,eps
0,119820,兄弟 結局彩蛋,fistfightpostendingscene_119820,1
1,115736,網戰,thewarnet_115736,1
2,111709,抓緊現在,mothersday2017image_111709,1
3,107119,愛情來的時候 新加坡,atimeoflovesingapore_107119,1
4,116348,降魔的番外篇-首部曲,thetaxorcistsidequedchapter1jade_116348,1
5,135816,曙光,silverlining_135816,1
6,135817,神奇的燈泡,youlightmeup_135817,1
7,135818,我的驕傲,mypride_135818,1
8,135819,母親的乒乓球,mymommypingpong_135819,1
9,135820,究竟天有幾高,touchthesky_135820,1


In [ ]:
#drop the programmes with only one episode
mytvdetails = [x for x in mytvdetails if x.get('latest_episode_no')>1]

In [ ]:
#filter the only required info from fetched data
mytvdetails_basic=[(
    x.get('programme_id')
    ,x.get('name_tc')
    ,x.get('path')
    ,x.get('latest_episode_no')
    ) for x in mytvdetails]

In [ ]:
df_mytvdetails=pd.DataFrame(mytvdetails_basic,columns=['id', 'name', 'path', 'eps'])
df_mytvdetails

,id,name,path,eps
0,123904,舌劍上的公堂,returnofthesilvertongue0002_123904,25
1,109568,巾幗梟雄之諜血長天 (26集電視版),noreserve0001_109568,26
2,102405,點解阿Sir係阿Sir,yessirsorrysir_102405,30
3,118791,是咁的，法官閣下,omgyourhonour_118791,25
4,104972,倩女喜相逢,myspiritualexlover_104972,15
...,...,...,...,...
275,124398,多功能老婆,wonderwomen0002_124398,25
276,133105,把關者們,thelinewatchers0001_133105,26
277,112626,燦爛的外母,thetofuwar_112626,20
278,139768,叠影狙擊,deadringer0001_139768,24


Some programme names contain unnecessary infomation in parentheses and should be removed

In [ ]:
df_mytvdetails[df_mytvdetails['name'].str.contains("\(")]

,id,name,path,eps
1,109568,巾幗梟雄之諜血長天 (26集電視版),noreserve0001_109568,26
21,131624,欺詐劇團 (全套上架),fraudstars_131624,8
23,113195,老表，畢業喇！ (最新集數免費重溫),ohmygrad_113195,30
90,106664,愛．回家 (#601-700),comehomelove601700_106664,700
91,106665,愛．回家 (#701-804),comehomelove701804_106665,804
94,106668,愛．回家 (#196-600),comehomelove196600_106668,600
98,126129,拆局專家 (最新集數免費重溫),thefixer0004_126129,21
107,137922,愛．回家 (#805-995),comehomelove805995_137922,995
118,117463,當旺爸爸 (最新集數免費重溫),daddygooddeeds0001_117463,20
123,130203,愛美麗狂想曲 (全套上架),beautyandtheboss_130203,30


The feature of normalize function:  
1. Remove the unwanted suffix
2. Normalize the Unicode characters
3. Remove all leading/trailing special characters
4. Trimming leading/trailing space


In [ ]:
remove_suffix = ['(最新集數免費重溫)','(全套上架)']
punct ='.,:;/-()「」'
def name_normalize(name):
  for sfx in remove_suffix:
    name = re.sub(re.escape(sfx)+'$', '', name)
  name = unicodedata.normalize("NFKC", name)
  name = re.sub(f'\s+(?=[{re.escape(punct)}])', '', name)
  name = re.sub(f'(?<![^{re.escape(punct)}])\s+', '', name)
  name = name.strip()
  return name

In [ ]:
df_mytvdetails['name'] = df_mytvdetails['name'].apply(name_normalize)

The sub category of programmes can be obtained from tags

In [ ]:
mytv_tags=[x.get('tags') for x in mytvdetails]

In [ ]:
#droping common tags ('and_tags', e.g. the 300-劇集 tag) for programmes
mytv_sub_category=[list(filter(lambda tag: tag.get('type')=="sub_category" and tag.get('tag_id') not in and_tags, tags)) for tags in mytv_tags]

In [ ]:
#function to get the Chinese name from a tag ID
def getnametc(lst):
  for idx in range(len(lst)):
    lst[idx]=[x.get('name_tc') for x in lst[idx]]
  return lst

In [ ]:
#convert the tag ID to Chinese names
mytv_sub_category=getnametc(mytv_sub_category)

In [ ]:
#flatten the programme ID & tag name records
df_sub_category=pd.DataFrame({'id':df_mytvdetails['id'],'sub_category':mytv_sub_category})
df_sub_category=df_sub_category.explode("sub_category").drop_duplicates().dropna()
df_sub_category

,id,sub_category
0,123904,劇情
0,123904,喜劇
0,123904,古裝
0,123904,刑偵
0,123904,法理
...,...,...
278,139768,動作
279,131581,劇情
279,131581,喜劇
279,131581,時裝


In [ ]:
#checking the unique sub_categories in record
df_sub_category['sub_category'].unique()

array(['劇情', '喜劇', '古裝', '刑偵', '法理', '民初', '倫理', '情感', '間諜', '時裝',
       '校園/青春', '勵志', '台慶劇', '奇幻', '武打', '特效', '動作', '警匪', '復仇', '黑暗',
       '寫實', '商戰', '歷史', '愛情', '行業', '歌舞', '廉政系列', '美食', '醫療', '奇情', '宮廷',
       '輕鬆', '靈異', '堪輿', '清裝', '溫情', '傳記', '幫會', '武俠', '香港回歸系列', '穿越',
       '寵物', '文教', '恐怖', '懸疑', '法醫', '鉅著', '處境劇'], dtype=object)

In [ ]:
#get the production year information of programme
mytv_prod_year=[list(filter(lambda tag: tag.get('type')=="prod_year", tags)) for tags in mytv_tags]

In [ ]:
#It was found that two records had more than one production year tag, representing the production start/end years
print(len(list(filter(lambda x :len(x)!=1, mytv_prod_year))))

2


In [ ]:
#convert the tag ID to Chinese names
mytv_prod_year=getnametc(mytv_prod_year)

In [ ]:
#create the production start/end columns for programme
df_mytvdetails['prod_start']=[min(x) for x in mytv_prod_year]
df_mytvdetails['prod_end']=[max(x) for x in mytv_prod_year]
df_mytvdetails[df_mytvdetails['prod_start']!=df_mytvdetails['prod_end']]

,id,name,path,eps,prod_start,prod_end
94,106668,愛.回家(#196-600),comehomelove196600_106668,600,2012,2014
177,133449,星空下的仁醫,kidslivesmatterhongkongversion_133449,25,2020,2021


In [ ]:
#create a flatten programme ID and actor record
mytv_actor=[x.get('artists') for x in mytvdetails]
mytv_actor=getnametc(mytv_actor)
df_actor=pd.DataFrame({'id':df_mytvdetails['id'],'actor':mytv_actor})
df_actor=df_actor.explode('actor').drop_duplicates().dropna()
df_actor

,id,actor
0,123904,郭晉安
0,123904,田蕊妮
0,123904,麥長青
0,123904,黃翠如
0,123904,林曉峰
...,...,...
279,131581,敖嘉年
279,131581,賴慰玲
279,131581,鄭子誠
279,131581,黃子雄


The "path" record can divided into three parts:  
1. <font color='red'>pathname</font>
2. <font color='blue'>number before underline</font> (can be null)
3. <font color='green'>number after underline</font>

For example
<font color='red'>noreserve</font><font color='blue'>0001</font><font color='green'>_109568</font>

The name part is used to fetch the programme/casting from TVB.com  
e.g.
https://programme.tvb.com/api/programme/detail?programme_path=threesome

However, some programme name on TVB.com is followed by a number, and that number may not be consistent with the <font color='blue'>number before underline</font> part of "path" record on mytv.com  
https://programme.tvb.com/api/programme/detail?programme_path=fraudstars0002  
If a record cannot be fetched with a <font color='red'>pathname</font>, the application will try to add 0001/0002/0003/0004 after the <font color='red'>pathname</font> and fetch again. The record will be marked as failed if it no data were fetched after the above attempts.

In [ ]:
#extracting the "pathname" part of path
df_mytvdetails['path']=df_mytvdetails['path'].str.split(r'(.+?)(00[0-9][1-9])?_', expand=True)[1]

It was found that the are duplicated "**pathname**" in records. The matching of mytv.com and tvb.com records will consider both "**pathname**" and "**name**"

In [ ]:
df_mytvdetails[df_mytvdetails['path'].duplicated(keep=False)].sort_values(by=['path'])

,id,name,path,eps,prod_start,prod_end
1,109568,巾幗梟雄之諜血長天(26集電視版),noreserve,26,2012,2012
149,108827,巾幗梟雄之諜血長天(30集原裝版),noreserve,30,2012,2012
181,135504,雙生陌生人(25集版),strangeranniversary,25,2022,2022
268,135134,雙生陌生人,strangeranniversary,20,2022,2022
198,116573,三個女人一個「因」(足本版),threesome,20,2016,2016
201,111459,三個女人一個「因」,threesome,9,2016,2016


It is also found two records with duplicated "**name**"

In [ ]:
df_mytvdetails[df_mytvdetails['name'].duplicated(keep=False)]

,id,name,path,eps,prod_start,prod_end
19,110117,拳王,glovescomeoff,25,2011,2011
56,133752,拳王,theringmaster,25,2020,2020


In [ ]:
#getting the unique "pathname" from records
unique_path=list(df_mytvdetails['path'].unique())
len(unique_path)

277

Below is the code for fetching programme details from tvb.com, it iterates through the "unique_path" list to find the matching records. Three types of error will be logged:  
1. Record fetched from "pathname" but they are records with duplicate "pathname"+"name" (error_log_duplicate)
2.Record fetched from "pathname" but the programme name from tvb.com cannot match any "name" records from mytv.com (error_log_mismatch)
3. Record cannot be fetched from "pathname" (error_log_notexist)

The matching record will be transformed and adding a "id" field from respective entry of mytv.com. This will be easier for later joining operations.

In [ ]:
error_log_duplicate=[]
error_log_mismatch=[]
error_log_notexist=[]

def fetchtvbcom(path):
  suffix_list=["",*['000'+str(x) for x in range(1,5)]]
  for sfx in suffix_list:
    details_url = "https://programme.tvb.com/api/programme/detail"
    query = {'programme_path': path+sfx}
    response = json.loads(requests.get(details_url,query).text)
    if response.get('code')==0:
      response['data']['tc_title'] = name_normalize(response['data']['tc_title'])
      title = name_normalize(response['data']['tc_title'])
      id = df_mytvdetails[((df_mytvdetails['name']==title) & (df_mytvdetails['path'].str.startswith(path, na=False)))]['id'].values
      if len(id)>=2:
        error_log_duplicate.append(title)
        print(f"Duplicated programme names in dataframe, cannot determine the ID of fetched data (title:{title})")
      elif len(id)==1:
        response['data']['id']= id[0].item()
        return response['data']
      else:
        error_log_mismatch.append(
            {"programme_title": title,
             "path": path
            }
        )
        print(f"programme found but the name does not exist in the dataframe (title:{title}, query path:{path})")
  error_log_notexist.append(path)
  print(f"programme not found (path:{path})")
  return response['data']

tvbdetail=[fetchtvbcom(x) for x in unique_path]
tvbdetail=[x for x in tvbdetail if x!=None]



with open('error_duplicate.log', 'w', encoding='utf-8') as f:
    json.dump(error_log_duplicate, f, ensure_ascii=False, indent=4)
with open('error_log_mismatch.log', 'w', encoding='utf-8') as f:
    json.dump(error_log_mismatch, f, ensure_ascii=False, indent=4)
with open('error_notexist.log', 'w', encoding='utf-8') as f:
    json.dump(error_log_notexist, f, ensure_ascii=False, indent=4)


programme not found (path:laundrystories)
programme not found (path:achangeofheart)
programme not found (path:theimpossible3)
programme not found (path:eyeinthesky)
programme not found (path:tigercubsii)
programme not found (path:comehomelove601700)
programme not found (path:comehomelove701804)
programme not found (path:comehomelove196600)
programme not found (path:murderdiary)
programme not found (path:ourunwindingethosmini)
programme not found (path:comehomelove805995)
programme not found (path:linewalkerthepreludecharacters)
programme not found (path:astepintothepastrecutversion)
programme not found (path:seasonoflove2013)
programme not found (path:atimeoflove2germanytrailer)
programme not found (path:theexecutioner)
programme not found (path:neighborhoodghoststories)
programme not found (path:forensicheroesiii)
programme found but the name does not exist in the dataframe (title:神探孖住上, query path:taketwo)
programme not found (path:immaria)
programme not found (path:raisingthebar)
pr

In [ ]:
# list of programmes that were on mytv.com but cannot be mapped to records on tvb.com
df_mytvdetails_mapped = df_mytvdetails.copy()
df_mytvdetails_mapped['foundontvb']=df_mytvdetails['id'].isin([x['id'] for x in mytvdetail])
df_mytvdetails_mapped[~df_mytvdetails_mapped['foundontvb']]

,id,name,path,eps,prod_start,prod_end,foundontvb
47,107109,洗衣鋪群星事件簿,laundrystories,14,2016,2016,False
58,117375,好心作怪,achangeofheart,30,2012,2012,False
77,128659,非凡三俠,theimpossible3,6,2020,2020,False
78,126107,天眼,eyeinthesky,20,2014,2014,False
86,116382,飛虎 II,tigercubsii,10,2013,2013,False
90,106664,愛.回家(#601-700),comehomelove601700,700,2014,2014,False
91,106665,愛.回家(#701-804),comehomelove701804,804,2015,2015,False
94,106668,愛.回家(#196-600),comehomelove196600,600,2012,2014,False
99,132271,刑偵日記,murderdiary,25,2020,2020,False
106,122557,十二傳說大謎蹤,ourunwindingethosmini,5,2018,2018,False


In [ ]:
#Showing the previously found records that have duplicated "pathname"
#It was found that only one of them having matching "name" on tvb.com
df_mytvdetails_mapped[df_mytvdetails_mapped['path'].duplicated(keep=False)].sort_values(by=['foundontvb'],ascending=False)

,id,name,path,eps,prod_start,prod_end,foundontvb
1,109568,巾幗梟雄之諜血長天(26集電視版),noreserve,26,2012,2012,True
201,111459,三個女人一個「因」,threesome,9,2016,2016,True
268,135134,雙生陌生人,strangeranniversary,20,2022,2022,True
149,108827,巾幗梟雄之諜血長天(30集原裝版),noreserve,30,2012,2012,False
181,135504,雙生陌生人(25集版),strangeranniversary,25,2022,2022,False
198,116573,三個女人一個「因」(足本版),threesome,20,2016,2016,False


In [ ]:
#Dropping the above records that were not found on tvb.com
df_mytvdetails_mapped=df_mytvdetails_mapped.sort_values(by=['foundontvb'],ascending=False).groupby('path').first().reset_index()

In [ ]:
df_mytvdetails_mapped[df_mytvdetails_mapped['path'].str.startswith("comehomelove", na=False)].sort_values(by=['foundontvb'],ascending=False)

,path,id,name,eps,prod_start,prod_end,foundontvb
50,comehomelove805995,137922,愛.回家(#805-995),995,2015,2015,True
46,comehomelove,128902,愛.回家(#1-195),195,2012,2012,False
47,comehomelove196600,106668,愛.回家(#196-600),600,2012,2014,False
48,comehomelove601700,106664,愛.回家(#601-700),700,2014,2014,False
49,comehomelove701804,106665,愛.回家(#701-804),804,2015,2015,False


In [ ]:
df_mytvdetails_mapped[df_mytvdetails_mapped['path'].str.startswith("loandbehold", na=False)].sort_values(by=['foundontvb'],ascending=False)

,path,id,name,eps,prod_start,prod_end,foundontvb
130,loandbehold,110561,愛.回家之開心速遞,2167,2023,2023,True
131,loandbeholdottversion,113042,愛.回家之開心速遞(網上版),218,2017,2017,False


In [ ]:
#After some mannul checking, it was found that 2 series were splitted by episodes on mytv.com
#However only one of each records was mapped to tvb.com
#Only the records that were able to mapped to tvb.com were kept
series_name=["loandbehold",'comehomelove']
for s_name in series_name:
  series_chk = df_mytvdetails_mapped[df_mytvdetails_mapped['path'].str.startswith(s_name, na=False)]
  if series_chk['foundontvb'].sum()==1:
    filter_series = series_chk.sort_values(by=['foundontvb'], ascending=False)['id'].iloc[1:]
    df_mytvdetails_mapped=df_mytvdetails_mapped[~df_mytvdetails_mapped['id'].isin(filter_series)]
  elif series_chk['foundontvb'].sum()>1:
    print(f'More than one details are found for the series {s_name}, please determine which one to keep')
  else:
    print((f'No details found for series {s_name}'))

In [ ]:
#By mannual checking, it was found that some programme's production year were not in the between 2010-2023, but still included in the result from mytv.com
# e.g. 133874:尋秦記(精華版)
#Those records were dropped mannually
drop_id=[133874,106760,106841,107109,113346,106956,106445,106958,122557,117168]
df_mytvdetails_mapped=df_mytvdetails_mapped[~df_mytvdetails_mapped['id'].isin(drop_id)]

In [ ]:
#programmes that were unable to map to tvb.com after cleaning
df_mytvdetails_mapped[~df_mytvdetails_mapped['foundontvb']]

,path,id,name,eps,prod_start,prod_end,foundontvb
1,achangeofheart,117375,好心作怪,30,2012,2012,False
15,armedreaction2021,130970,陀槍師姐2021,30,2020,2020,False
66,eyeinthesky,126107,天眼,20,2014,2014,False
72,forensicheroesiii,110908,法證先鋒III,30,2010,2010,False
80,ghettojusticeii,109469,怒火街頭 2,21,2011,2011,False
144,murderdiary,132271,刑偵日記,25,2020,2020,False
156,neighborhoodghoststories,119095,區區有鬼故,7,2014,2014,False
181,raisingthebar,110440,四個女仔三個BAR,25,2014,2014,False
194,seasonoflove2013,103156,戀愛季節,20,2013,2013,False
223,theexecutioner,126260,刀下留人,26,2015,2015,False


In [ ]:
#filtering the actor/sub_category DF to include results from cleanned programme list only
df_actor = df_actor[df_actor['id'].isin(df_mytvdetails_mapped['id'])]
df_sub_category = df_sub_category[df_sub_category['id'].isin(df_mytvdetails_mapped['id'])]

In [ ]:
#Mannually backfill some programmes that have expectional pathnames
backfill_list=[
    (126107,'eyeinthesky2015')
    ,(130970,'armedreaction2021hongkongversion')
    ,(109469,"ghettojustice2")
    ,(132271,"murderdiaryhongkongversion")
    ,(126260,"executioner")
    ,(112008,"droppingbycloudnine")
]

def tvb_backfill(id, path):
  details_url = "https://programme.tvb.com/api/programme/detail"
  query = {'programme_path': path}
  response = json.loads(requests.get(details_url,query).text)
  response['data']['id']= id
  return response['data']

mytvdetail_mapped=mytvdetail
for id, path in backfill_list:
  mytvdetail_mapped.append(tvb_backfill(id, path))

In [ ]:
#create an actor list from the fetched data of tvb.com
df_tvb_actor=pd.DataFrame({
    'id': [x['id'] for x in mytvdetail_mapped]
    ,'actor': [[tc_name['tc_artiste_name'] for tc_name in x['cast']] for x in mytvdetail_mapped]
})
df_tvb_actor=df_tvb_actor.explode('actor')
df_tvb_actor=df_tvb_actor.dropna()

In [ ]:
#merging the actor list from tvb.com into the actor list from mytv.com, dropping the duplicates
df_actor = pd.concat([df_tvb_actor,df_actor],ignore_index=True).drop_duplicates()

In [ ]:
#getting the crew list from fetched data of tvb.com
df_crew=pd.DataFrame({
    'id': [x['id'] for x in mytvdetail_mapped]
    ,'crew': [[ (k,[name['tc_name'] for name in v]) for k,v in x['crew'].items() ] for x in mytvdetail_mapped]
})

In [ ]:
df_crew=df_crew.explode('crew').dropna()
df_crew

,id,crew
0,123904,"(administrative_assistant, [周慕妍])"
0,123904,"(executive_producer, [李艷芳])"
0,123904,"(producer, [吳超榮, 陳湘娟, 鄭永潮])"
0,123904,"(production_assistant, [張永豪, 鄺錦宏, 李鳳明, 馮肇均, 陳曉..."
0,123904,"(supervisor, [劉枝華])"
...,...,...
254,112008,"(executive_producer, [何小慧])"
254,112008,"(producer, [陳志華, 梁志康, 楊任豪, 葉登偉, 蔡健光])"
254,112008,"(production_assistant, [周雲, 何佩欣, 湯琳貞, 姚啟衡, 潘琳貞..."
254,112008,"(supervisor, [徐家祥])"


In [ ]:
#rearrange and flatten the crew record
df_crew['post']=df_crew['crew'].map(lambda x:x[0])
df_crew['name']=df_crew['crew'].map(lambda x:x[1])
df_crew=df_crew.explode('name')
df_crew=df_crew.drop(columns=['crew']).drop_duplicates().dropna()
df_crew

,id,post,name
0,123904,administrative_assistant,周慕妍
0,123904,executive_producer,李艷芳
0,123904,producer,吳超榮
0,123904,producer,陳湘娟
0,123904,producer,鄭永潮
...,...,...,...
254,112008,supervisor,徐家祥
254,112008,writer,龍文康
254,112008,writer,黃熙麗
254,112008,writer,陳錦雯


In [ ]:
#fetching the casting list from tvb.com, using "pathnames" that sucessfully fetch the program details from tvb.com
def fetchcast(path):
  cast_url = "https://programme.tvb.com/api/programme/cast"
  query = {'programme_path':path}
  return [(x['tc_artiste_name'],x['tc_cast_name']) for x in json.loads(requests.get(cast_url,query).text)['data']['list']]

cast_list = [fetchcast(x['programme_path']) for x in mytvdetail_mapped]

In [ ]:
#creating a DF for the fetched casting data
df_cast=pd.DataFrame({
    'id': [x['id'] for x in mytvdetail_mapped]
    ,'cast': cast_list
})

In [ ]:
#extract actor & role from the fetched results
df_cast=df_cast.explode('cast').dropna()
df_cast['actor']=df_cast['cast'].map(lambda x:x[0])
df_cast['role']=df_cast['cast'].map(lambda x:x[1])
df_cast=df_cast.drop(columns=['cast'])

In [ ]:
#filter casting with empty roles and actor name equals to his/her role
df_cast=df_cast[(df_cast['actor']!=df_cast['role'])&(df_cast['role']!="")]

Joining the actors lists:  
1. Actor (without role) information from mytv.com (previously joined)
2. Actor (without role) information from tvb.com (previously joined)
3. Actor & Role information from tvb.com

In [ ]:
df_cast_all=pd.concat([df_cast,df_actor],ignore_index=True)

In [ ]:
#showing non-duplicated records
df_cast_all[df_cast_all.duplicated(subset=['id','actor'],keep=False)]

,id,actor,role
0,123904,郭晉安,章四維
1,123904,田蕊妮,陳真真
2,123904,麥長青,夏侯武
3,123904,黃翠如,周菊
4,123904,林曉峰,潘爾名
...,...,...,...
22437,112008,莊思敏,NaN
22441,112008,高可慧,NaN
22463,112008,何雁詩,NaN
22464,112008,許廷鏗,NaN


In [ ]:
#normalize the strings of role/actor
for i in ['role','actor']:
  df_cast_all[i]=df_cast_all[i].apply(lambda x: unicodedata.normalize("NFKC", str(x)).strip())

In [ ]:
#drop duplicated record, keeping first = keeping role information (if any) (it was concatenate before actor (without role) information)
df_cast_all=df_cast_all.drop_duplicates(subset=['id','actor'], keep='first')

Handling the irregularity in data  
Some programmes will have the same actor label as "actor" and "actor1"  
e.g. "黎耀祥" & "黎耀祥1" with different roles

In [ ]:
df_cast_all[df_cast_all['actor'].str.contains("[0-9]$")]

,id,actor,role
14,109568,黎耀祥1,鈴木一雄
1313,126538,劉蔚萱1,賓客
1314,126538,何廣沛1,選手
1315,126538,黃耀英1,選手
1316,126538,譚永浩1,選手
...,...,...,...
9905,110524,唐嘉麟1,禮手下
9906,110524,方紹聰1,Simon
16033,105197,謝可逸1,NaN
16390,111363,黃智賢1,NaN


In [ ]:
#Removing the numbers acter actor name
df_cast_all['actor']=df_cast_all['actor'].apply(lambda x :re.sub(f'[0-9]+$', '', x))

In [ ]:
#some role records are marked with "/"
df_cast_all[df_cast_all['role'].str.contains('/', na=False)]

,id,actor,role
98,128025,李豪,豪/街坊
102,128025,沈愛琳,琳/街坊
115,128025,潘冠霖,笑/職員
125,128025,柯嵐,手下/喜
130,128025,趙樂賢,手下/恭
...,...,...,...
10698,126107,陳俊堅,飛虎隊/乘客
10726,126107,焦浩軒,鋒/飛虎隊/乘客
10730,126107,盧偉文,記者/飛虎隊
10735,126107,阮浩棕,泳客/乘客


In [ ]:
#flatten the record with roles marked with '/'
df_cast_all['role']=df_cast_all['role'].str.split("/")
df_cast_all=df_cast_all.explode('role')

In [ ]:
#trimming and drop duplicated records
df_cast_all['role']=df_cast_all['role'].apply(lambda x: x.strip() if type(x)==str else x)
df_cast_all=df_cast_all.drop_duplicates()

In [ ]:
#grouping roles in a list for the actor in same programme ID
df_cast_grouped=df_cast_all.groupby(['id','actor'], sort=False)['role'].apply(lambda x: list(x[x.notna()]))
df_cast_grouped=df_cast_grouped.reset_index()
df_cast_grouped

,id,actor,role
0,123904,郭晉安,[章四維]
1,123904,田蕊妮,[陳真真]
2,123904,麥長青,[夏侯武]
3,123904,黃翠如,[周菊]
4,123904,林曉峰,[潘爾名]
...,...,...,...
11719,125904,岑麗香,[]
11720,125904,蘇玉華,[]
11721,125904,袁偉豪,[]
11722,125904,黃淑儀,[]


The below programmes cannot find any actor (& role) information

In [ ]:
df_mytvdetails_mapped[(~df_mytvdetails_mapped['id'].isin(df_cast_grouped['id']))]

,path,id,name,eps,prod_start,prod_end,foundontvb
37,brotherskeeper2,105360,巨輪II,39,2015,2015,True
108,ipccspecial,133393,監警最前線,4,2021,2021,True
153,myspiritualexlover,104972,倩女喜相逢,15,2014,2014,True
194,seasonoflove2013,103156,戀愛季節,20,2013,2013,False
201,silverspoonsterlingshackles,102425,名媛望族,40,2011,2011,True


The below programmes cannot find any crew information

In [ ]:
df_mytvdetails_mapped[(~df_mytvdetails_mapped['id'].isin(df_crew['id']))]

,path,id,name,eps,prod_start,prod_end,foundontvb
1,achangeofheart,117375,好心作怪,30,2012,2012,False
58,deadringer,139768,叠影狙擊,24,2023,2023,True
67,fashionwar,105197,潮流教主,20,2015,2015,True
72,forensicheroesiii,110908,法證先鋒III,30,2010,2010,False
108,ipccspecial,133393,監警最前線,4,2021,2021,True
111,k9cop,105759,警犬巴打,20,2015,2015,True
138,mayfortunesmileonyou,109821,財神駕到,30,2015,2015,True
142,moderndynasty,134534,家族榮耀,30,2021,2021,True
156,neighborhoodghoststories,119095,區區有鬼故,7,2014,2014,False
181,raisingthebar,110440,四個女仔三個BAR,25,2014,2014,False


In [ ]:
#mannually backfill some actor information found on the Internet
manual_backfill=pd.DataFrame([[105360, ["蕭正楠","田蕊妮","陳凱琳", "楊明","吳岱融","阮兆祥","梁琤","徐榮", "李佳芯","韋家雄","趙永洪","張致恆"] ,[]]
              ,[133393, ["陳國邦","蔣祖曼","張松枝","雷深如","李明憲","李紫僖"], []]
              ,[104972, ["薛家燕","蕭正楠","麥長青","梁競徽","滕麗名","湯盈盈","吳若希","于洋","鄧梓峰","安德尊"], []]
              ,[103156, ["羅仲謙","梁靖琪","徐子珊","吳卓羲","胡定欣","王浩信","梁烈唯","馬國明","胡杏兒"], []]
              ,[102425, ["劉松仁","陳玉蓮", "楊怡", "馬國明","吳卓羲","朱晨麗","韓馬利","江美儀","王浩信","黃智賢","馬賽","周志文","賈曉晨","李思欣","劉江","石修"], []]
              ],columns=['id', 'actor', 'role'])

In [ ]:
manual_backfill=manual_backfill.explode('actor')
df_cast_grouped=pd.concat([df_cast_grouped,manual_backfill],ignore_index=True)
df_cast_grouped

,id,actor,role
0,123904,郭晉安,[章四維]
1,123904,田蕊妮,[陳真真]
2,123904,麥長青,[夏侯武]
3,123904,黃翠如,[周菊]
4,123904,林曉峰,[潘爾名]
...,...,...,...
11772,102425,周志文,[]
11773,102425,賈曉晨,[]
11774,102425,李思欣,[]
11775,102425,劉江,[]


In [ ]:
df_mytvdetails_mapped[df_mytvdetails_mapped['name'].str.contains('\(')]

,path,id,name,eps,prod_start,prod_end,foundontvb
50,comehomelove805995,137922,愛.回家(#805-995),995,2015,2015,True
162,noreserve,109568,巾幗梟雄之諜血長天(26集電視版),26,2012,2012,True


In [ ]:
#Remove some trailing information in the name of programme for consistency
df_mytvdetails_mapped['name']=df_mytvdetails_mapped['name'].apply(lambda x: re.sub("\s*\(.+\)$","",x))

In [ ]:
#Export the final results
with open('df_cast.json', 'w', encoding='utf-8') as file:
    df_cast_grouped.to_json(file, force_ascii=False)

with open('df_details.json', 'w', encoding='utf-8') as file:
    df_mytvdetails_mapped.drop(columns=['foundontvb']).to_json(file, force_ascii=False)

with open('df_crew.json', 'w', encoding='utf-8') as file:
    df_crew.reset_index(drop=True).to_json(file, force_ascii=False)

with open('df_sub_category.json', 'w', encoding='utf-8') as file:
    df_sub_category.reset_index(drop=True).to_json(file, force_ascii=False)